In [1]:
import json
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import datetime
import os
from pycocotools import coco
import xml.etree.ElementTree as ET
import xml
from pycocotools import mask as comask
import cv2
from itertools import groupby
from utils import DateCaptured, coco_bbox_creator, xml_retreiver, iou_calculator, biggest_segment, mask2poly, binary_mask_to_rle

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
### CATEGORY INFORMATION

# Add the name of the folders inside the dataset and
# assign some label id. Start iding from 91.

dict_of_categories = {'face': 91}

In [3]:
### DATASET INFORMATION

# Enter the original/custom JSON file to be used.
# I used the original train2014 JSON file containing
# the instances.

json_dir = "Datasets/coco/annotations_old/instances_train2014.json"

### For viewing the content of the new json file. ###
# json_dir = "Datasets/coco/annotations/instances_train2014.json"

### minival2014 will become the main dictionary ###
minival2014 = json.load(open(json_dir))

In [4]:
### CLEARING THE DATASET
# Use that if you only going to make a dataset of your own.
# If you are going to append the images, this operation is not necessary.

# minival2014['images'] = []
# minival2014['annotations'] = []

In [5]:
### APPENDING CLASSES TO THE CATEGORIES

### For viewing the categories inside the JSON file.
# minival2014['categories']

# Add the classes based on to the dict_of_categories
# Below shows an example for adding the face class to the main dictionary. 

minival2014['categories'].append({'id': 91, 'name': 'face', 'supercategory': 'person'})



In [14]:
# Some functions to be used:
def isjpg(string):
    if string[-4:]==".jpg":
        return True 
    
def ispng(string):
    if string[-4:]==".png":
        return True

In [13]:
# LabelMe type
dataset_dir = "/home/mspr/Desktop/Mask_RCNN/Datasets/"  # Will be changed with argparse, includes directories for video names and corresponding frames inside
dataset_name = "MSPR_Dataset/"

target_dataset_name = os.path.join(dataset_dir, "coco/train2014/")

##########################################################################
##########################################################################
##########################
# TO DO, MATCHING AND CALCULATING IOU'S BETWEEN OZAN'S AND FILIZ'S ANNOTATIONS
##########################
avg_iou = []

##########################
# ANNOTATIONS OF FILIZ   #
##########################
filiz_annotations = os.path.join("/home/mspr/Desktop/Mask_RCNN/Datasets/MSPR_Dataset/face/filiz_Annotations",
                                 "pascal_voc_face annotation.txt")

# image_nrs, bbox_filiz = txt_bbox_parser(filiz_annotations)
##########################################################################
##########################################################################

# Added image and annotation ids will start from 600k. 
image_id = 600000
annotation_id = 600000

for class_name in dict_of_categories.keys():
    annot_all_files = sorted(os.listdir(os.path.join(dataset_dir, dataset_name, class_name, annot_dir)))
    for xml_file in annot_all_files:
        annot_dir = os.path.join(dataset_dir, dataset_name, class_name, "Annotations/")
        mask_dir = os.path.join(dataset_dir, dataset_name, class_name, "Masks/")
        image_dir = os.path.join(dataset_dir, dataset_name, class_name, "Images/")
        new_mark_dir = os.path.join(dataset_dir, dataset_name, class_name, "Masks_GT/")

        tree = ET.parse(os.path.join(annot_dir, xml_file))
        root = tree.getroot()

        delete_flags = root.findall("./object/deleted")
        sum_deleted = sum(map(int, [delete_flag.text for delete_flag in delete_flags]))

        xmins = root.findall("./object/segm/box/xmin")
        ymins = root.findall("./object/segm/box/ymin")
        xmaxs = root.findall("./object/segm/box/xmax")
        ymaxs = root.findall("./object/segm/box/ymax")

        mask_filenames = root.findall("./object/segm/mask")
        # IMAGES
        # Image will be copied to MS COCO train directory and file name will be hold
        image_filename = xml_retreiver(root, 'filename')
        image = io.imread(os.path.join(image_dir, image_filename))
        if image_id<1000000:
            image_target_filename = "COCO_train2014_000000"+str(image_id)+".jpg"
        elif (image_id>=1000000 or image_id<=10000000):
            image_target_filename = "COCO_train2014_00000"+str(image_id)+".jpg"

# Do not rewrite the image if something exists
###TODO: read the image and rewrite it if the content and target are not the same
#         if os.path.isfile(os.path.join(target_dataset_name, image_target_filename))==False:
#             io.imsave(os.path.join(target_dataset_name, image_target_filename), image)
        
        io.imsave(os.path.join(target_dataset_name, image_target_filename), image)

        # Image height and width
        height = int(xml_retreiver(root, 'nrows'))
        width = int(xml_retreiver(root, 'ncols'))

        assert height == image.shape[0]
        assert width == image.shape[1]


        # Miscellaneous metadata
        date_captured = DateCaptured()
        coco_url = 'n/a'
        flickr_url = 'n/a'
        license = np.random.randint(8)

        # Appending to 'images'
        minival2014['images'].append({'coco_url': coco_url, 'file_name': image_target_filename, 
                                     'date_captured': date_captured, 'flickr_url': flickr_url,
                                     'height': height, 'id': image_id, 'license': license, 
                                     'width': width})
        # Polygon Handling

        for d, delete_flag in enumerate(delete_flags):
            delete_flag = int(delete_flag.text)
            if not delete_flag:


                # ANNOTATIONS
                # Bbox
                bbox_x = int(xmins[d].text)
                bbox_y = int(ymins[d].text)
                bbox_w = int(xmaxs[d].text) - bbox_x
                bbox_h = int(ymaxs[d].text) - bbox_y
                bbox_list = [bbox_x, bbox_y, bbox_w, bbox_h]

##########################################################################
##########################################################################
#                 queries_findid = [d for d, image_nr in enumerate(image_nrs) if image_nr==image_filename[:-4]]
#                 for query in queries_findid:
#                     print(bbox_filiz[query])
#                     print(bbox_list)
#                 iou_for_each_xml_object = [iou_calculator(bbox_filiz[query], bbox_list) for query in queries_findid]
#                 avg_iou.append(iou_calculator(bbox_filiz[query], bbox_list) for query in queries_findid)    
#                 print(iou_for_each_xml_object)
#                 print("")


#                 print(bbox_x, bbox_y, bbox_w, bbox_h)

#                 with open("bbox_info.txt", "a+") as f:
#                     f.write(image_filename[:-4]+","+str(bbox_x)+","+str(bbox_y)+","+
#                             str(bbox_x+bbox_w)+","+str(bbox_y)+","+
#                             str(bbox_x)+","+str(bbox_y+bbox_h)+","+
#                            str(bbox_x+bbox_w)+","+str(bbox_y+bbox_h)+"\n")

##########################################################################
##########################################################################

                # Class
                category_id = dict_of_categories[class_name]

                # Masks Segmentation
                instance_seg_info = []
                mask = cv2.imread(os.path.join(mask_dir, mask_filenames[d].text), 0)
                polygons, binary_image = mask2poly(mask)
                cv2.imwrite(os.path.join(new_mark_dir, mask_filenames[d].text), binary_image*255)
                RLE_mask = binary_mask_to_rle(mask)
                RLE_mask_coco = comask.encode(np.asfortranarray(binary_image))

                # Area
                area = comask.area(RLE_mask_coco)

                #  Iscrowd
                if (len(mask_filenames) - sum_deleted) > 1:
                    iscrowd = 1
                    minival2014['annotations'].append({'iscrowd': iscrowd, 'bbox': bbox_list, 'id': annotation_id,
                                                       'image_id': image_id, 'segmentation': RLE_mask,
                                                       'area': area, 'category_id': dict_of_categories[class_name]})
                else:
                    iscrowd = 0
                    minival2014['annotations'].append({'iscrowd': iscrowd, 'bbox': bbox_list, 'id': annotation_id,
                                                  'image_id': image_id, 'segmentation': polygons,
                                                  'area': area, 'category_id': dict_of_categories[class_name]})

                # Appending into annotations
                
                annotation_id+=1
        image_id+=1

                
#                 print(bbox_x, bbox_y, bbox_w, bbox_h)
# print("Average IoU = {}".format(sum(avg_iou)/len(avg_iou)))

In [21]:
# Storing the data in memory and then writes them into a file. 

# from decimal import Decimal
from json import encoder
encoder.FLOAT_REPR = lambda o: format(o, '.2f')

class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

with open("Datasets/coco/annotations/instances_train2014.json","w") as f:
# with open("Datasets/coco/annotations/instances_valminusminival2014.json","w") as f:
    data = json.dumps(minival2014, cls=MyEncoder, indent=4)
    f.write(data)
